In [2]:
import yfinance as yf
import pandas as pd
#from pandas import DataFrame
import itertools
import numpy as np
import statistics as stat
import dask
from dask.distributed import Client, LocalCluster
import os
#import csv
import requests
from bs4 import BeautifulSoup
from getuseragent import UserAgent

In [3]:
saveas_path = '/Users/Nawar/Documents/Travaux/Invest Tool/Extracts'
MDDs_path = '/Users/Nawar/Documents/Travaux/Invest Tool/Analysis/Tickers/'
file_output_type = '.csv'
tickers_file = '/Users/Nawar/Documents/Travaux/Invest Tool/Scopes/S&P 500 Components - mkt cap > 3000.csv'
df_tickers = pd.read_csv(tickers_file, sep = ',')
starting_date = '2022-11-30'
ending_date = '2023-03-09'
concat_file_name = "Output from " + starting_date + " to " + ending_date
concat_file_path = '/Users/Nawar/Documents/Travaux/Invest Tool/Analysis/Outputs/'

In [37]:
def ticker_info(my_saveas_path, my_MDD_path, my_ticker, my_filename, my_file_output_type):

    #directory_in_str = my_saveas_path
    #directory = os.fsencode(directory_in_str)
    #df_MDDs = pd.DataFrame()

    #for file in os.listdir(directory):
    
    #filename = os.fsdecode(file)
    ticker = my_ticker
    #print(my_MDD_path + ticker + my_file_output_type)

    try:
        info = pd.DataFrame()#yf.Ticker(ticker).info
        #name = info['shortName']
        #ask = info['ask']

    except:
        #name = ''
        #ask = ''
        info = pd.DataFrame()

    if my_filename.endswith('.csv'):

        filefullname = my_saveas_path + '/Prices/' + my_filename
        #print(filefullname)

    try:
        df = pd.read_csv(filefullname)
        #print(info)
        PEG_analysis = Function_PEG(ticker)
        MDD = Function_MaxDrawnDown(df)
        Volatility = Function_Volatility(df)
        #data_info = pd.DataFrame({'Ticker': ticker, 'Name': name, "Ask Price": ask}, index =[0])
        #data_info = pd.DataFrame([info])
        #data_info = pd.DataFrame(info, index = [0])
        #df.drop('a', inplace=True, axis=1)

        #data_info = data_info.drop('longBusinessSummary', axis=1)

        #except:
         #   data_info = pd.DataFrame(info, index = [0])
        #print(MDD)
        #my_data = pd.concat([Volatility, MDD, data_info], axis=1, sort=True)
        #my_data = pd.concat([PEG_analysis, Volatility, MDD, data_info], axis=1, sort=True)
        my_data = pd.concat([PEG_analysis, Volatility, MDD], axis=1, sort=True)
        df_MDD = pd.DataFrame(my_data)
        df_MDD.to_csv(my_MDD_path + ticker + my_file_output_type, index = False)
        #df_MDDs = df_MDDs.append(df_MDD, ignore_index=True)

    except:
        print('Data error for',ticker)

In [5]:
def Function_MaxDrawnDown(my_df):

    MDD = 0
    data = {'MDD':'no MDD', 'Occurrence':'no occurrence', 'Max Price':'no MaxPrice'} 
    
    for index in range(1, len(my_df)):
        gap = my_df.loc[len(my_df)-1,'Adj Close']/my_df.loc[len(my_df) - index,'Adj Close'] - 1
        if gap < MDD:
            
            MDD = gap
            occurrence = my_df.loc[len(my_df)-index,'Date']
            MaxPrice = my_df.loc[len(my_df)-index,'Adj Close']
            data = {'MDD':["{:.2%}".format(MDD)], 'Occurrence':[occurrence], 'Max Price':[MaxPrice]} 
    
    output = pd.DataFrame(data, index = [0])
    
    #output = ("{:.2%}".format(MDD),occurrence)
    
    return output

In [6]:
def Function_Volatility(my_df):
    
    Vol_Vector = pd.DataFrame()
    try:
        for index in range(0, len(my_df)-1):

            Dayli_Perf = my_df.loc[index + 1,'Close']/my_df.loc[index,'Close'] - 1
            Dayli_Perf = {'Dayli_Perf':[Dayli_Perf]} 
            Dayli_Perf = pd.DataFrame(Dayli_Perf, index = [index])
            #print(Dayli_Perf)

            #Vol_Vector = Vol_Vector.append(Dayli_Perf)#, ignore_index=True)
            Vol_Vector = pd.concat([Vol_Vector, Dayli_Perf])#, axis=1)#, sort=True)

        #print(Vol_Vector)
        #Vol = stat.stdev(Vol_Vector)
        Vol = Vol_Vector['Dayli_Perf'].std()
        data = {'Volatility':['{:.2%}'.format(Vol)]} 
    except:
        data = {'Volatility':9999999} 
    
    output = pd.DataFrame(data)#, index = [0])
    
    return output

In [35]:
def Function_PEG(my_ticker):
    
#     #earnings_f0
#     try:
#         df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Earnings/'+my_ticker+'/Earnings History.csv', index_col=0, header=0).loc[['EPS Actual']]

#         try:
#             df = pd.to_numeric(df.iloc[0])
#         except:
#             df = pd.to_numeric(df.iloc[0],errors='coerce')
#             print(my_ticker,': NA value in Earnings History file on EPS Actual -> earnings_f0 calculation not accurate')

#         earnings_f0 = df.sum()

#     except:
#         earnings_f0 = 'NA'
#         print(my_ticker,': in Earnings History file "EPS Actual" keyword not found -> earnings_f0 missing')
    
    
    
    #earnings_f0, earnings_f1 and earnings_f2
    df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Earnings/'+my_ticker+'/Earnings Estimate.csv', index_col=0, header=0)
    column_names = list(df.columns.values)
    line_names = list(df.index)

    if 'Avg. Estimate' in str(line_names):
        Avg_estimate_index = len(tuple(itertools.takewhile(lambda x: "Avg. Estimate" not in x, line_names)))
        #earnings_f1
        if 'Current Year' in str(column_names):
            current_year_index = len(tuple(itertools.takewhile(lambda x: "Current Year" not in x, column_names)))
            earnings_f1 = df.iloc[Avg_estimate_index,current_year_index]
            #nb_analysts_f1
            if 'No. of Analysts' in str(line_names):
                nb_analysts_index = len(tuple(itertools.takewhile(lambda x: "No. of Analysts" not in x, line_names)))
                nb_analysts_f1 = df.iloc[nb_analysts_index,current_year_index]
            else:
                nb_analysts_f1 = 'NA'
            #low_estimate_f1
            if 'Low Estimate' in str(line_names):
                low_estimate_index = len(tuple(itertools.takewhile(lambda x: "Low Estimate" not in x, line_names)))
                low_estimate_f1 = df.iloc[low_estimate_index,current_year_index]
            else:
                low_estimate_f1 = 'NA'
            #high_estimate_f1
            if 'High Estimate' in str(line_names):
                high_estimate_index = len(tuple(itertools.takewhile(lambda x: "High Estimate" not in x, line_names)))
                high_estimate_f1 = df.iloc[high_estimate_index,current_year_index]
            else:
                high_estimate_f1 = 'NA'
            #standard_dev_eps_f1
            try:
                standard_dev_eps_f1 = np.std([low_estimate_f1, high_estimate_f1])
            except:
                standard_dev_eps_f1 = 'NA'
            #earnings_f0
            if 'Year Ago EPS' in str(line_names):
                earnings_f0_index = len(tuple(itertools.takewhile(lambda x: "Year Ago EPS" not in x, line_names)))
                earnings_f0 = df.iloc[earnings_f0_index,current_year_index]
            else:
                earnings_f0 = 'NA'
        else:
            earnings_f1 = 'NA'
            print(my_ticker,': in Earnings Estimate file "Current Year" keyword not found -> earnings_f1 missing')
        #earnings_f2    
        if 'Next Year' in str(column_names):
            next_year_index = len(tuple(itertools.takewhile(lambda x: "Next Year" not in x, column_names)))
            earnings_f2 = df.iloc[Avg_estimate_index,next_year_index]
            #nb_analysts_f2
            if 'No. of Analysts' in str(line_names):
                nb_analysts_index = len(tuple(itertools.takewhile(lambda x: "No. of Analysts" not in x, line_names)))
                nb_analysts_f2 = df.iloc[nb_analysts_index,next_year_index]
            else:
                nb_analysts_f2 = 'NA'
            #low_estimate_f2
            if 'Low Estimate' in str(line_names):
                low_estimate_index = len(tuple(itertools.takewhile(lambda x: "Low Estimate" not in x, line_names)))
                low_estimate_f2 = df.iloc[low_estimate_index,next_year_index]
            else:
                low_estimate_f2 = 'NA'
            #high_estimate_f2
            if 'High Estimate' in str(line_names):
                high_estimate_index = len(tuple(itertools.takewhile(lambda x: "High Estimate" not in x, line_names)))
                high_estimate_f2 = df.iloc[high_estimate_index,next_year_index]
            else:
                high_estimate_f2 = 'NA'
            #standard_dev_eps_f2
            try:
                standard_dev_eps_f2 = np.std([low_estimate_f2, high_estimate_f2])
            except:
                standard_dev_eps_f2 = 'NA'
        else:
            earnings_f2 = 'NA'
            print(my_ticker,': in Earnings Estimate file "Next Year" keyword not found -> earnings_f2 missing')
            
    else:
        earnings_f1 = 'NA'
        earnings_f2 = 'NA'
        print(my_ticker,': in Earnings Estimate file "Avg. Estimate" keyword not found -> earnings_f1/earnings_f2 missing')
        
        
        
    #growth_f1 and growth_f2
    if not earnings_f1 == 'NA':
        
        if not earnings_f0 == 'NA' and not earnings_f0 == 0:
            try:
                growth_f1 = earnings_f1/earnings_f0-1
            except:
                growth_f1 = 'NA'
                print(my_ticker,': growth_f1 missing')
        else:
            growth_f1 = 'NA'

        if not earnings_f2 == 'NA' and not earnings_f1 == 0:
            try:
                growth_f2 =  earnings_f2/earnings_f1-1
            except:
                growth_f2 = 'NA'
                print(my_ticker,': growth_f2 missing')
        else:
            growth_f2 = 'NA'
            
    else:
        growth_f1 = 'NA'
        growth_f2 = 'NA'
        print(my_ticker,': growth_f1/growth_f2 missing')
    

    
    #current_price
    try:
        df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Prices/'+my_ticker+'.csv', index_col=0, header=0)
        #df = yf.Ticker(my_ticker).history()
        #current_price = yf.Ticker(my_ticker).info['currentPrice']
        current_price = df['Close'].iloc[-1]
    except:
        current_price = 'NA'
        print(my_ticker,': current price missing')
    
    
    
    #sharesOutstanding
    try:
        end_period_share = 'NA'#yf.Ticker(my_ticker).info['sharesOutstanding']
    except:
        end_period_share = 'NA'
        print(my_ticker,': shares outstanding missing')
        
        
        
    #sector & industry & fiscal month
    df = pd.read_csv(tickers_file, index_col=0, header=0, sep = ',')
    column_names = list(df.columns.values)
    line_names = list(df.index)
    # if my_ticker in str(line_names):
    try:
        my_ticker_index = len(tuple(itertools.takewhile(lambda x: my_ticker not in x, line_names)))

        #sector
        # if 'Sector' in str(column_names):
        try:
            sector_index = len(tuple(itertools.takewhile(lambda x: "Sector" not in x, column_names)))
            sector = df.iloc[my_ticker_index,sector_index]
        except:
            sector = "NA"
            
    
        #industry
        # if 'Industry' in str(column_names):
        try:
            industry_index = len(tuple(itertools.takewhile(lambda x: "Industry" not in x, column_names)))
            industry = df.iloc[my_ticker_index,industry_index]
        except:
            industry = "NA"
            
        #fiscal month
        # if 'Month of Fiscal Yr End' in str(column_names):
        try:
            fiscal_month_index = len(tuple(itertools.takewhile(lambda x: "Month of Fiscal Yr End" not in x, column_names)))
            fiscal_month = df.iloc[my_ticker_index,fiscal_month_index]
        except:
            fiscal_month = "NA"
            
    except:
        sector = "NA"
        industry = "NA"
        fiscal_month = "NA"
            
    
    
    #pe_f1 and pe_f2
    if not current_price == 'NA':

        if not earnings_f0 == 'NA' and not earnings_f0 == 0:
            try:
                pe_f0 = current_price/earnings_f0
            except:
                pe_f0 = 'NA'
            print(my_ticker,': pe_f0 missing')
        else:
            pe_f0 = 'NA'
            
        if not earnings_f1 == 0:
            try:  
                pe_f1 = current_price/earnings_f1
            except:
                pe_f1 = 'NA'
                print(my_ticker,': pe_f1 missing')
        else:
            pe_f1 = 'NA'

        if not earnings_f2 == 0:    
            try: 
                pe_f2 = current_price/earnings_f2
            except:
                pe_f2 = 'NA'
                print(my_ticker,': pe_f2 missing')
        else:
            pe_f2 = 'NA'

    else:
        pe_f1 = 'NA'
        pe_f2 = 'NA'
        print(my_ticker,': pe_f1/pe_f2 missing')
    
    
    
    #peg_f1 and peg_f2
    try:
    # if not pe_f1 == 'NA' and not growth_f1 == 'NA' and not growth_f1 == 0:
        peg_f1 = pe_f1/growth_f1
    except:
        print(my_ticker,': peg_f1 missing')
        peg_f1 = 'NA'
        
    try:
    # if not pe_f1 == 'NA' and not growth_f2 == 'NA' and not growth_f2 == 0:
        peg_f2 = pe_f2/growth_f2
    except:
        print(my_ticker,': peg_f2 missing')
        peg_f2 = 'NA'
    
    
    
    #surprise average
    try:
        df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Earnings/'+my_ticker+'/Earnings History.csv', index_col=0, header=0).loc[['Difference']]
        df = pd.to_numeric(df.iloc[0])
        surprise_average = df.mean()
    except:
        surprise_average = 'NA'
    
    
    
    #fiscal month
    # try:
    #     df = yf.Ticker(my_ticker).balance_sheet
    #     fiscal_date = list(df.columns.values)[0]
    #     fiscal_month = str(np.datetime64(fiscal_date, 'M'))[-2:]
    # except:
    #     fiscal_month = 'NA'
        
    data = {'Sector':[sector], 'Industry':[industry], 'Ticker':[my_ticker], 'Earnings F0':[earnings_f0], 'Earnings F1':[earnings_f1], 'Earnings F2':[earnings_f2], 'Growth F1':[growth_f1],
            'Growth F2':[growth_f2], 'PE F0':[pe_f0], 'PE F1':[pe_f1], 'PE F2':[pe_f2], 'PEG F1':[peg_f1], 'PEG F2':[peg_f2], 'Surprise Average':[surprise_average],
            'No. of analysts f1':[nb_analysts_f1], 'No. of analysts f2':[nb_analysts_f2], 'Standard dev eps f1':[standard_dev_eps_f1], 'Standard dev eps f2':[standard_dev_eps_f2],
            'Fiscal Month':[fiscal_month]} 
    output = pd.DataFrame(data, index = [0])
    
    return output

In [8]:
def Extract(my_ticker, my_starting_date, my_ending_date, my_saveas_path, my_file_output_type):
    try:
        data = yf.download(my_ticker, start = starting_date, end = ending_date)
        data.to_csv(my_saveas_path + '/Prices/' + my_ticker + my_file_output_type)
        #Event().wait(5) 
    except:
        print(str(my_ticker) + ' extract failed : prices')
    

In [9]:
def Extract_Earnings(my_ticker, my_saveas_path, my_file_output_type):
    my_url = 'https://finance.yahoo.com/quote/' + str(my_ticker) + '/analysis?p=' + str(my_ticker)
    #https://finance.yahoo.com/quote/NA/analysis?p=NA
    useragent = UserAgent()
    theuseragent = useragent.Random()
    my_headers = {'User-Agent': theuseragent}
    response = requests.get(my_url, headers = my_headers)

    soup = BeautifulSoup(response.text, "lxml")
    tables = soup.find_all('table')
    number_tables = len(tables)
    try:
        read_response = pd.read_html(response.text)
    
        try:
            os.mkdir(my_saveas_path + '/Earnings/'+ my_ticker)
        except:
            pass

        try:
            for index in range(0, number_tables):
                df = pd.DataFrame(read_response[index])
                table_name = df.columns[0]
                df.to_csv(my_saveas_path + '/Earnings/'+ my_ticker + '/' + table_name + my_file_output_type, index = False)
        except:
            print(str(my_ticker) + ' extract failed : earnings')
            try:
                os.remove(my_saveas_path + '/Earnings/'+ my_ticker)
            except:
                pass
    except:
        print(my_ticker,': no earnings tables found')

In [10]:
def Extract_Profile(my_ticker, my_saveas_path, my_file_output_type):
    my_url = 'https://finance.yahoo.com/quote/' + str(my_ticker) + '/profile?p=' + str(my_ticker)
    #https://finance.yahoo.com/quote/CLX/profile?p=CLX
    useragent = UserAgent()
    theuseragent = useragent.Random()
    my_headers = {'User-Agent': theuseragent}
    response = requests.get(my_url, headers = my_headers)

    soup = BeautifulSoup(response.text, "lxml")
    tables = soup.find_all('table')
    number_tables = len(tables)
    try:
        read_response = pd.read_html(response.text)
    
        try:
            os.mkdir(my_saveas_path + '/Profile/'+ my_ticker)
        except:
            pass

        try:
            for index in range(0, number_tables):
                df = pd.DataFrame(read_response[index])
                table_name = df.columns[0]
                df.to_csv(my_saveas_path + '/Profile/'+ my_ticker + '/' + table_name + my_file_output_type, index = False)
        except:
            print(str(my_ticker) + ' extract failed : profile')
            try:
                os.remove(my_saveas_path + '/Profile/'+ my_ticker)
            except:
                pass
    except:
        print(my_ticker,': no profile tables found')

In [11]:
#extraction loop

dask_df_list = []

with LocalCluster() as cluster, Client(cluster) as client:
    for index, row in df_tickers.iterrows():

        ticker = row[0]

        dask_extract = dask.delayed(Extract_Earnings)(ticker, saveas_path, file_output_type)#ok
        #Extract_Earnings(ticker, saveas_path, file_output_type)#ok

        dask_df_list.append(dask_extract)

    dask.compute(dask_df_list)

for index, row in df_tickers.iterrows():
        
    ticker = row[0]

    #dask_extract = dask.delayed(Extract)(ticker, starting_date, ending_date, saveas_path, file_output_type)#nok
    Extract(ticker, starting_date, ending_date, saveas_path, file_output_type)#ok

#dask_concat_df = dask.delayed(concat_df)(dask_df_list, folder_dir)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [38]:
#MDD loop

directory_in_str = saveas_path + '/Prices'
directory = os.fsencode(directory_in_str)
#df_MDDs = pd.DataFrame()

dask_df_list = []

#for index, row in df_tickers.iterrows():
for file in os.listdir(directory):
        
        filename = os.fsdecode(file)
        ticker = filename[0:len(filename)-4]
        #print(filename, ticker)
        dask_extract = dask.delayed(ticker_info)(saveas_path, MDDs_path, ticker, filename, file_output_type)
        #def ticker_info(my_saveas_path, my_MDD_path, my_ticker, my_filename, my_file_output_type):
        #ticker_info(saveas_path, MDDs_path, ticker, filename, file_output_type)
        
        dask_df_list.append(dask_extract)
        #dask_df_list.concat(dask_extract)

#dask_concat_df = dask.delayed(concat_df)(dask_df_list, folder_dir)

In [39]:
%%time

data = dask.compute(dask_df_list)
print('All tasks completed')

AVY AQUA: pe_f0 missing
PEN : pe_f0 missing
 : pe_f0 missing
OTIS APD: pe_f0 missing : pe_f0 missing

BRP VIPS : pe_f0 missingAGL: pe_f0 missing 
: pe_f0 missing

AGL : peg_f2 missing
VRTX : pe_f0 missing


<ipython-input-35-da7f079831af>:231: RuntimeWarning: divide by zero encountered in double_scalars
  peg_f1 = pe_f1/growth_f1


TRU : pe_f0 missing
MTCH : pe_f0 missingMELI
 RITM: pe_f0 missingEVRYOU   : pe_f0 missing: pe_f0 missing: pe_f0 missing


MMS
 : pe_f0 missing
AMT : pe_f0 missing
WTFC : pe_f0 missing
LBTYK SQM : pe_f0 missing
PSTG: pe_f0 missing 
: pe_f0 missing
LBTYK : peg_f1 missing
LBTYK : peg_f2 missing
INGRCMCSA  : pe_f0 missing
: pe_f0 missing
EOG : pe_f0 missing
NKE NEP : pe_f0 missing
Data error for CWEN
: pe_f0 missing
GOOGL : pe_f0 missing
J : pe_f0 missing
IP : pe_f0 missing
HLI : pe_f0 missing
GRMN : pe_f0 missing
ACN : pe_f0 missing
ABBVFC : pe_f0 missing
 : pe_f0 missing
CERT : pe_f0 missing
BCPC : pe_f0 missing
STLD : pe_f0 missing
PEAK : pe_f0 missing
PNR : pe_f0 missing
MDB : pe_f0 missing
Data error for IWP
UNMFUL : pe_f0 missing
 : pe_f0 missing
BGNE : pe_f0 missing
OLED : pe_f0 missing
QFIN EQIX: pe_f0 missing
 IBP : pe_f0 missing
: pe_f0 missing
CCEP : pe_f0 missing
DHI : pe_f0 missing
SCI : pe_f0 missingKMX
 : pe_f0 missing
IAC : pe_f0 missing
AZPNData error for EWJ
 WIRE: pe_f0 

<ipython-input-35-da7f079831af>:231: RuntimeWarning: divide by zero encountered in double_scalars
  peg_f1 = pe_f1/growth_f1


LBRDA : pe_f0 missing
KLIC : pe_f0 missing
CGNX : pe_f0 missing
SNN : pe_f0 missing
SNN : peg_f1 missing
SNN : peg_f2 missing
DG : pe_f0 missing
TS : pe_f0 missing
BMI : pe_f0 missing
SYNH : pe_f0 missing
FISV : pe_f0 missing
ACT : pe_f0 missing
DCI : pe_f0 missing
FCNAOS : pe_f0 missing
 : pe_f0 missing
MLM : pe_f0 missing
FWONK : pe_f0 missing
ALKS : pe_f0 missing
SNAP : pe_f0 missing
BAM : pe_f0 missing
SEM : pe_f0 missing
CMS : pe_f0 missing
MANH : pe_f0 missing
CMI : pe_f0 missing
BLKBSQ : pe_f0 missing
 : pe_f0 missing
LYFT : pe_f0 missing
KEPMMM  : pe_f0 missing
: pe_f0 missing
KEP : peg_f1 missing
KEP : peg_f2 missing
DGX : pe_f0 missing
TROW : pe_f0 missingHAS
 : pe_f0 missing
CRH PCTY : pe_f0 missing
: pe_f0 missing
CRH : peg_f1 missing
CRH : peg_f2 missing
SCCO : pe_f0 missing
Data error for AGG
REGN : pe_f0 missing


...

Data error forOWL BRK.B
 : pe_f0 missing
TFSL : pe_f0 missing
RDNRGA : pe_f0 missing
 : pe_f0 missing
HSIC : pe_f0 missing
MGA : pe_f0 missing
BLDR RLI : pe_f0 missing
: pe_f0 missing
Data error for IYW
KRC : pe_f0 missing
ONTO SNV: pe_f0 missing 
ONTO : peg_f2 missing
: pe_f0 missing
FUTU : pe_f0 missing
GSK : pe_f0 missing
VRSKAIT : pe_f0 missing
 : pe_f0 missing
NTRS : pe_f0 missing
KHCELV  : pe_f0 missing: pe_f0 missing

ES : pe_f0 missing
ESMT : pe_f0 missing
LITE : pe_f0 missing
RYN : pe_f0 missing


<ipython-input-35-da7f079831af>:238: RuntimeWarning: divide by zero encountered in double_scalars
  peg_f2 = pe_f2/growth_f2


PPG ALV: pe_f0 missing : pe_f0 missing

WPC : pe_f0 missing
ESGR : pe_f0 missing
ABNB : pe_f0 missing
OC : pe_f0 missing
BK : pe_f0 missing
HEI SGRY: pe_f0 missing : pe_f0 missing

ZLAB NVS: pe_f0 missing
 : pe_f0 missing
STN : pe_f0 missing
AAPL : pe_f0 missing
PWSC : pe_f0 missing
SNDR : pe_f0 missing
EIX : pe_f0 missing
PKX : pe_f0 missing
PKX : peg_f1 missing
PKX : peg_f2 missing
WEC : pe_f0 missing
ATR : pe_f0 missing


<ipython-input-35-da7f079831af>:231: RuntimeWarning: divide by zero encountered in double_scalars
  peg_f1 = pe_f1/growth_f1


LAZ : pe_f0 missing
KKR : pe_f0 missing
YETI : pe_f0 missing
GDWFRD  : pe_f0 missing: pe_f0 missing

CHTR : pe_f0 missing
Data error forFIXNNI  : pe_f0 missing
 : pe_f0 missing
ENIC
BIDU : pe_f0 missing
ENSG : pe_f0 missingAER : pe_f0 missing

PXD : pe_f0 missing
ITT : pe_f0 missing
MSCI : pe_f0 missing
OILMN : pe_f0 missing
 : pe_f0 missing
QS : pe_f0 missing
GWREZSFM : pe_f0 missing
  : pe_f0 missing: pe_f0 missing

Data error for IGIB
MCHP : pe_f0 missing
VST : pe_f0 missing
SONY KO: pe_f0 missing : pe_f0 missing

MRCY : pe_f0 missing
NYTFNV  : pe_f0 missing: pe_f0 missing

MSA : pe_f0 missing
GRFS : pe_f0 missing
GRFS : peg_f1 missing
GRFS : peg_f2 missing
Data error for CTRA
Data error for IVV
IFF : pe_f0 missing
ABCM : pe_f0 missing
ABCM : peg_f1 missing
ABCM : peg_f2 missing
GPI : pe_f0 missing
ADBE : pe_f0 missing
PIIGGB  : pe_f0 missing
: pe_f0 missingEMN 
: pe_f0 missing
PODD : pe_f0 missing
AWI : pe_f0 missingESS
 : pe_f0 missing
KRG : pe_f0 missing
Data error for IJS
DLB : 

In [ ]:
#Concatanate file

directory_in_str = MDDs_path
directory = os.fsencode(directory_in_str)
#df_MDDs = pd.DataFrame()

#df_single = pd.DataFrame()
df_concat = pd.DataFrame()

#for index, row in df_tickers.iterrows():
for file in os.listdir(directory):
        
        filename = os.fsdecode(file)
        ticker = filename[1:len(filename)-4]

        if filename.endswith('.csv'):
            filefullname = directory_in_str + '/' + filename
            df_single = pd.read_csv(filefullname)
            #df_concat = df_concat.append(df_single, ignore_index=True, sort=False)
            df_concat = pd.concat([df_concat, df_single], ignore_index=True, sort=False)
            

df_concat.to_csv(concat_file_path + concat_file_name + '.csv', index = False)

In [41]:
#Concatanate file
#Adding delta PE vs Bench fields in concat file

directory_in_str = MDDs_path
directory = os.fsencode(directory_in_str)
#df_MDDs = pd.DataFrame()

#df_single = pd.DataFrame()
df_concat = pd.DataFrame()

#for index, row in df_tickers.iterrows():
for file in os.listdir(directory):
        
    filename = os.fsdecode(file)
    ticker = filename[1:len(filename)-4]

    if filename.endswith('.csv'):
        filefullname = directory_in_str + '/' + filename
        df_single = pd.read_csv(filefullname)
        #df_concat = df_concat.append(df_single, ignore_index=True, sort=False)
        df_concat = pd.concat([df_concat, df_single], ignore_index=True, sort=False)

df_concat['Earnings F0 Industry Bench'] = df_concat['Earnings F0'].groupby(df_concat['Industry']).transform('mean')
df_concat['Earnings F1 Industry Bench'] = df_concat['Earnings F1'].groupby(df_concat['Industry']).transform('mean')
df_concat['Earnings F2 Industry Bench'] = df_concat['Earnings F2'].groupby(df_concat['Industry']).transform('mean')
df_concat['Earnings F0 Sector Bench'] = df_concat['Earnings F0'].groupby(df_concat['Sector']).transform('mean')
df_concat['Earnings F1 Sector Bench'] = df_concat['Earnings F1'].groupby(df_concat['Sector']).transform('mean')
df_concat['Earnings F2 Sector Bench'] = df_concat['Earnings F2'].groupby(df_concat['Sector']).transform('mean')

df_concat['Earnings F0 Delta industry'] = df_concat['Earnings F0'] - df_concat['Earnings F0 Industry Bench']
df_concat['Earnings F1 Delta industry'] = df_concat['Earnings F1'] - df_concat['Earnings F1 Industry Bench']
df_concat['Earnings F2 Delta industry'] = df_concat['Earnings F2'] - df_concat['Earnings F2 Industry Bench']
df_concat['Earnings F0 Delta sector'] = df_concat['Earnings F0'] - df_concat['Earnings F0 Sector Bench']
df_concat['Earnings F1 Delta sector'] = df_concat['Earnings F1'] - df_concat['Earnings F1 Sector Bench']
df_concat['Earnings F2 Delta sector'] = df_concat['Earnings F2'] - df_concat['Earnings F2 Sector Bench']
        

#df_concat['Growth F0 Industry Bench'] = df_concat['Growth F0'].groupby(df_concat['Industry']).transform('mean')
df_concat['Growth F1 Industry Bench'] = df_concat['Growth F1'].groupby(df_concat['Industry']).transform('mean')
df_concat['Growth F2 Industry Bench'] = df_concat['Growth F2'].groupby(df_concat['Industry']).transform('mean')
#df_concat['Growth F0 Sector Bench'] = df_concat['Growth F0'].groupby(df_concat['Sector']).transform('mean')
df_concat['Growth F1 Sector Bench'] = df_concat['Growth F1'].groupby(df_concat['Sector']).transform('mean')
df_concat['Growth F2 Sector Bench'] = df_concat['Growth F2'].groupby(df_concat['Sector']).transform('mean')

#df_concat['Growth F0 Delta industry'] = df_concat['Growth F0'] - df_concat['Growth F0 Industry Bench']
df_concat['Growth F1 Delta industry'] = df_concat['Growth F1'] - df_concat['Growth F1 Industry Bench']
df_concat['Growth F2 Delta industry'] = df_concat['Growth F2'] - df_concat['Growth F2 Industry Bench']
#df_concat['Growth F0 Delta sector'] = df_concat['Growth F0'] - df_concat['Growth F0 Sector Bench']
df_concat['Growth F1 Delta sector'] = df_concat['Growth F1'] - df_concat['Growth F1 Sector Bench']
df_concat['Growth F2 Delta sector'] = df_concat['Growth F2'] - df_concat['Growth F2 Sector Bench']

    
df_concat['PE F0 Industry Bench'] = df_concat['PE F0'].groupby(df_concat['Industry']).transform('mean')
df_concat['PE F1 Industry Bench'] = df_concat['PE F1'].groupby(df_concat['Industry']).transform('mean')
df_concat['PE F2 Industry Bench'] = df_concat['PE F2'].groupby(df_concat['Industry']).transform('mean')
df_concat['PE F0 Sector Bench'] = df_concat['PE F0'].groupby(df_concat['Sector']).transform('mean')
df_concat['PE F1 Sector Bench'] = df_concat['PE F1'].groupby(df_concat['Sector']).transform('mean')
df_concat['PE F2 Sector Bench'] = df_concat['PE F2'].groupby(df_concat['Sector']).transform('mean')

df_concat['PE F0 Delta industry'] = df_concat['PE F0'] - df_concat['PE F0 Industry Bench']
df_concat['PE F1 Delta industry'] = df_concat['PE F1'] - df_concat['PE F1 Industry Bench']
df_concat['PE F2 Delta industry'] = df_concat['PE F2'] - df_concat['PE F2 Industry Bench']
df_concat['PE F0 Delta sector'] = df_concat['PE F0'] - df_concat['PE F0 Sector Bench']
df_concat['PE F1 Delta sector'] = df_concat['PE F1'] - df_concat['PE F1 Sector Bench']
df_concat['PE F2 Delta sector'] = df_concat['PE F2'] - df_concat['PE F2 Sector Bench']

df_concat['PE F1 vs F0'] = df_concat['PE F1'] - df_concat['PE F0']
df_concat['PE F2 vs F1'] = df_concat['PE F2'] - df_concat['PE F1']

df_concat['Scoring F0'] = df_concat['Earnings F0 Delta industry'] + df_concat['Earnings F0 Delta sector'] + df_concat['PE F0 Delta industry'] + df_concat['PE F0 Delta sector']

df_concat['Scoring F1'] = df_concat['Earnings F1 Delta industry'] + df_concat['Earnings F1 Delta sector'] + df_concat['PE F1 Delta industry'] + df_concat['PE F1 Delta sector'] 
+ df_concat['Growth F1 Delta industry'] + df_concat['Growth F1 Delta sector'] + df_concat['No. of analysts f1'] + df_concat['PE F1 vs F0'] #+ df_concat['Standard dev eps f1']

df_concat['Scoring F2'] = df_concat['Earnings F2 Delta industry'] + df_concat['Earnings F2 Delta sector'] + df_concat['PE F2 Delta industry'] + df_concat['PE F2 Delta sector']
+ df_concat['Growth F2 Delta industry'] + df_concat['Growth F2 Delta sector'] + df_concat['No. of analysts f2'] + df_concat['PE F2 vs F1'] #+ df_concat['Standard dev eps f2'] 

#Surprise Average
df_concat['Surprise Average Industry Bench'] = df_concat['Surprise Average'].groupby(df_concat['Industry']).transform('mean')
df_concat['Surprise Average Sector Bench'] = df_concat['Surprise Average'].groupby(df_concat['Sector']).transform('mean')

#df_concat.drop('longBusinessSummary', inplace=True, axis=1)
        
df_concat.to_csv(concat_file_path + concat_file_name + '.csv', index = False)

In [ ]:
#Testing Zone

In [ ]:

dask_df_list = []

with LocalCluster(
    processes=True,
    threads_per_worker=1,
    memory_limit='auto'
) as cluster, Client(cluster) as client:
    for index, row in df_tickers.iterrows():
        
        ticker = row[0]

        dask_extract = dask.delayed(Extract)(ticker, starting_date, ending_date, saveas_path, file_output_type)#nok
        #Extract(ticker, starting_date, ending_date, saveas_path, file_output_type)#ok
        
        dask_df_list.append(dask_extract)
    
    dask.compute(dask_df_list)

In [ ]:
import requests
from bs4 import BeautifulSoup
from getuseragent import UserAgent

In [ ]:
my_url = 'https://finance.yahoo.com/quote/AAPL/analysis?p=AAPL'
useragent = UserAgent()
theuseragent = useragent.Random()
my_headers = {'User-Agent': theuseragent}
response = requests.get(my_url, headers = my_headers)

In [ ]:
soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all('table')
number_tables = len(tables)
read_response = pd.read_html(response.text)

for index in range(0, number_tables):
    df = pd.DataFrame(read_response[index])
    table_name = df.columns[0]
    df.to_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Earnings/'+ table_name + '.csv', index = False)

In [ ]:
read_response = pd.read_html(response.text)
df = pd.DataFrame(read_response[0], read_response[1])
df.to_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Earnings/'table.csv', index = False)

In [ ]:
df=yf.Ticker('AEO').history()
df['Close'].iloc[-1]

In [ ]:
data = yf.download('AEO', start = starting_date, end = ending_date)

In [ ]:
try:
    df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/Prices/AEO.csv', index_col=0, header=0)
    #df = yf.Ticker(my_ticker).history()
    current_price = df['Close'].iloc[-1]
except:
    current_price = 'NA'
    print(my_ticker,': current price missing')

In [ ]:
df = pd.read_csv('/Users/Nawar/Documents/Travaux/Invest Tool/Analysis/Outputs/Output from 2022-10-31 to 2023-01-27.csv')

In [ ]:
#df['PE_F1_Industry'] = df['PE F1'].groupby(df['industry'])
df['test'] = df.groupby(float(df['PE F1'])).apply(lambda x: np.average(x.industry, weights=float(x['No. of analysts f1'])))

In [ ]:
df['PE_F1_Industry'] = df['PE F1'].groupby(df['industry'])

In [ ]:
msft = yf.Ticker("MSFT")

# fast access to subset of stock info


In [ ]:
msft = yf.Ticker("acdc")
df = msft.balance_sheet
fiscal_date = list(df.columns.values)[0]
fiscal_month = str(np.datetime64(fiscal_date, 'M'))[-2:]

In [ ]:
fiscal_month

In [ ]:
help(yf.ticker)

In [ ]:
    
    #sharesOutstanding
    # try:
end_period_share = yf.Ticker("acdc").fast_info
    # except:
    #     end_period_share = 'NA'
        # print(my_ticker,': shares outstanding missing')
        
#     #sector
#     try:
#         sector = yf.Ticker(my_ticker).info['sector']
#     except:
#         sector = 'NA'
#         print(my_ticker,': sector missing')
        
#     #industry
#     try:
#         industry = yf.Ticker(my_ticker).info['industry']
#     except:
#         end_period_share = 'NA'
#         print(my_ticker,': industry missing')

In [ ]:
end_period_share

In [ ]:
saveas_path = '/Users/Nawar/Documents/Travaux/Invest Tool/Extracts/test.csv'
my_url = 'https://finance.yahoo.com/quote/CLX/profile?p=CLX'
useragent = UserAgent()
theuseragent = useragent.Random()
my_headers = {'User-Agent': theuseragent}
response = requests.get(my_url, headers = my_headers)

soup = BeautifulSoup(response.text, "lxml")
tables = soup.find_all('table')
number_tables = len(tables)

read_response = pd.read_html(response.text)

df = pd.DataFrame(read_response[0])
table_name = df.columns[0]
read_response.to_csv(saveas_path)


In [ ]:
read_response

In [ ]:
df = pd.read_csv(tickers_file, index_col=0, header=0, sep = ';')
column_names = list(df.columns.values)
line_names = list(df.index)
my_ticker = 'AAL'

if my_ticker in str(line_names):
    my_ticker_index = len(tuple(itertools.takewhile(lambda x: my_ticker not in x, line_names)))
    #sector
    if 'Sector' in str(column_names):
        sector_index = len(tuple(itertools.takewhile(lambda x: "Sector" not in x, column_names)))
        sector = df.iloc[my_ticker_index,sector_index]

In [ ]:
column_names